In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

$$\Large\text{Code}$$

In [12]:
import re

In [13]:
!ls ~/test-jupyter/P2/M2/SM2/logs

hadoop-hdfs-datanode-node.log  hadoop-hdfs-namenode-node.log


In [14]:
file_path1 = 'file:////home/talentum/test-jupyter/P2/M2/SM2/logs/hadoop-hdfs-datanode-node.log'
file_path2 = 'file:////home/talentum/test-jupyter/P2/M2/SM2/logs/hadoop-hdfs-namenode-node.log' 

In [15]:
def get_errors(file_path1, file_path2, timestamp_pattern=''):
    fileRDD = sc.textFile(file_path1).union(sc.textFile(file_path2))
    
    # lines with error and UnknownHostException in them
    fileRDD_filter_error_1 = fileRDD.filter(lambda line: (('ERROR' in line) or ('error' in line)) and 'UnknownHostException' in line)
    
    # lines with timestamp_patther
    fileRDD_filter_error_2 = fileRDD_filter_error_1.filter(lambda line: re.match(timestamp_pattern, line))

    print("The total number of lines with the keyword error is", fileRDD_filter_error_2.count())

    print('\n\nLines with error:')
    for line in fileRDD_filter_error_2.collect():
        print(line)

In [16]:
timestamp_pattern = r'2012-10-15 08:(1[0-5])'
get_errors(file_path1,file_path2, timestamp_pattern)

The total number of lines with the keyword error is 4


Lines with error:
2012-10-15 08:11:07,074 ERROR org.apache.hadoop.metrics2.sink.ganglia.GangliaSink31: java.net.UnknownHostException: node: node
2012-10-15 08:11:48,113 ERROR org.apache.hadoop.hdfs.server.datanode.DataNode: java.net.UnknownHostException: node: node
2012-10-15 08:11:07,069 ERROR org.apache.hadoop.metrics2.sink.ganglia.GangliaSink31: java.net.UnknownHostException: node: node
2012-10-15 08:11:49,328 ERROR org.apache.hadoop.hdfs.server.namenode.NameNode: java.net.UnknownHostException: Invalid hostname for server: node
